In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
# GPU configuration
!nvidia-smi

In [ ]:
# Install required packages
!pip install tensorflow==2.12.0 tensorflow-addons==0.20.0 keras==2.12.0
!pip install typeguard==2.13.3 inflect==6.0.2
!pip install visualkeras
!pip install numpy==1.23.5

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Define labels and image size
labels = ["Citrus canker", "Citrus greening", "Citrus mealybugs", "Die back", "Foliage damaged", "Healthy leaf", "Powdery mildew", "Shot hole", "Spiny whitefly", "Yellow dragon", "Yellow leaves"]
img_size = 32

In [ ]:
# Data loading function
def get_data(data_dir):
    data = []
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img))[...,::-1]
                resized_arr = cv2.resize(img_arr, (img_size, img_size))
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [ ]:
# Load and preprocess data
data_dir = get_data("/content/drive/Shareddrives/4IR_Research_Students/Emon/Sweet_orange")
X = np.array([i[0] for i in data_dir]) / 255.0
Y = np.array([i[1] for i in data_dir])

In [ ]:
# Split data
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.10, stratify=Y, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.18, random_state=8)

print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_val shape: {x_val.shape} - y_val shape: {y_val.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")

In [ ]:
def create_simple_cnn_model(input_shape):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5)
    ])
    return model


In [ ]:
def create_simple_vit_model(input_dim, num_classes=11):
    num_patches = 16
    projection_dim = 64
    num_heads = 8

    inputs = layers.Input(shape=(input_dim,))
    x = layers.Dense(num_patches * projection_dim)(inputs)
    x = layers.Reshape((num_patches, projection_dim))(x)

    # Single Transformer layer
    attn_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=projection_dim)(x, x)
    x = layers.Add()([x, attn_output])
    x = layers.LayerNormalization(epsilon=1e-6)(x)

    x = layers.Dense(projection_dim * 2, activation="gelu")(x)
    x = layers.Dense(projection_dim)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)

    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    return models.Model(inputs, outputs)

In [ ]:
def create_integrated_model(input_shape, num_classes=11):
    cnn_model = create_simple_cnn_model(input_shape)
    cnn_output = cnn_model.output_shape[1]
    vit_model = create_simple_vit_model(cnn_output, num_classes)

    inputs = layers.Input(shape=input_shape)
    x = cnn_model(inputs)
    outputs = vit_model(x)

    return models.Model(inputs, outputs)

In [ ]:
# Create and compile the model
input_shape = (32, 32, 3)
num_classes = 11
integrated_model = create_integrated_model(input_shape, num_classes)
integrated_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])


In [ ]:
# Callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=5),
]


In [ ]:
# Train the model
history = integrated_model.fit(
    data_augmentation(x_train),
    y_train,
    validation_data=(x_val, y_val),
    epochs=100,
    batch_size=32,
    callbacks=callbacks
)


In [ ]:
# Evaluate the model
train_loss, train_accuracy = integrated_model.evaluate(x_train, y_train)
test_loss, test_accuracy = integrated_model.evaluate(x_test, y_test)

print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Function to plot confusion matrix
def plot_confusion_matrix(y_true, y_pred, labels, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.ylabel('Actual Label')
    plt.xlabel('Predicted Label')
    plt.title(title)
    plt.tight_layout()
    plt.show()

In [ ]:
# Validation set predictions and metrics
y_val_pred = integrated_model.predict(x_val)
y_val_pred_classes = np.argmax(y_val_pred, axis=1)

print("Validation Set Classification Report:")
print(classification_report(y_val, y_val_pred_classes, target_names=labels))

plot_confusion_matrix(y_val, y_val_pred_classes, labels, 'Validation Set Confusion Matrix')

In [ ]:
# Test set predictions and metrics
y_test_pred = integrated_model.predict(x_test)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)

print("Test Set Classification Report:")
print(classification_report(y_test, y_test_pred_classes, target_names=labels))

plot_confusion_matrix(y_test, y_test_pred_classes, labels, 'Test Set Confusion Matrix')